# SQL query from table names

In This notebook we are going to test if using just the name of the table, and a shord definition of its contect we can use a model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

In [71]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [72]:
#Function to call the model.
def return_OAI(user_message):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)
    context = []
    context.append({'role':'system', "content": user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=context,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [73]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

In [74]:
print(text_tables)

Engineers: Engineer details: name, age, specialization, years of experience
Specialties: List of engineering specializations (e.g., Aerospace, Software, Mechanical, Civil)
Projects: Project details: project name, type, duration, assigned engineers
Companies: Company details: name, industry, number of employees, headquarters


In [75]:
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Questyion:
{question}
"""


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try a few versions if you have time
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

# Version One: Healthcare Data

In [76]:
import pandas as pd

# Table and definitions sample
data = {
    'table': ['Doctors', 'Patients', 'Branches', 'Hospitals'],
    'definition': [
        'Doctors information, name, age, specialty, years of experience',
        'Patient details, medical history, assigned doctor, year of admission',
        'Hospital branches, location, number of doctors, departments',
        'Hospital information, name, capacity, facilities, affiliations'
    ]
}

df = pd.DataFrame(data)
print(df)

df.head()


       table                                         definition
0    Doctors  Doctors information, name, age, specialty, yea...
1   Patients  Patient details, medical history, assigned doc...
2   Branches  Hospital branches, location, number of doctors...
3  Hospitals  Hospital information, name, capacity, faciliti...


,table,definition
0,Doctors,"Doctors information, name, age, specialty, yea..."
1,Patients,"Patient details, medical history, assigned doc..."
2,Branches,"Hospital branches, location, number of doctors..."
3,Hospitals,"Hospital information, name, capacity, faciliti..."


-  Convert Table Definitions to Text Format

In [77]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])
print(text_tables)

Doctors: Doctors information, name, age, specialty, years of experience
Patients: Patient details, medical history, assigned doctor, year of admission
Branches: Hospital branches, location, number of doctors, departments
Hospitals: Hospital information, name, capacity, facilities, affiliations


- Generate and Execute the First Query

In [78]:
#Creating the prompt, with the user questions and the tables definitions.
pqt1 = prompt_question_tables.format(tables=text_tables, question='Return the doctor ID of the patient with Patient_ID = 123')
print(return_OAI(pqt1))

```json
{
    "tables": ["Doctors", "Patients"]
}
```


- Generate the Second Query - aggregated result

In [79]:
pqt3 = prompt_question_tables.format(tables=text_tables,
                                     question='Return the doctor with the highest number of experience')
print(return_OAI(pqt3))

```json
{
    "tables": ["Doctors"]
}
```


# Version Two: Engineering Data

In [80]:
import pandas as pd

# Table and definitions
data = {
    'table': ['Engineers', 'Specialties', 'Projects','Companies'],
    'definition': [
        'Engineer details: name, age, specialization, years of experience',
        'List of engineering specializations (e.g., Aerospace, Software, Mechanical, Civil)',
        'Project details: project name, type, duration, assigned engineers',
        'Company details: name, industry, number of employees, headquarters'
    ]
}

df = pd.DataFrame(data)
print(df)

df.head()


         table                                         definition
0    Engineers  Engineer details: name, age, specialization, y...
1  Specialties  List of engineering specializations (e.g., Aer...
2     Projects  Project details: project name, type, duration,...
3    Companies  Company details: name, industry, number of emp...


,table,definition
0,Engineers,"Engineer details: name, age, specialization, y..."
1,Specialties,"List of engineering specializations (e.g., Aer..."
2,Projects,"Project details: project name, type, duration,..."
3,Companies,"Company details: name, industry, number of emp..."


-  Convert Table Definitions to Text Format

In [81]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])
print(text_tables)

Engineers: Engineer details: name, age, specialization, years of experience
Specialties: List of engineering specializations (e.g., Aerospace, Software, Mechanical, Civil)
Projects: Project details: project name, type, duration, assigned engineers
Companies: Company details: name, industry, number of employees, headquarters


- Generate and Execute the First Query

In [82]:
pqt1 = prompt_question_tables.format(tables=text_tables, question="based on performance and dedication, which engineering specialty is known for having the most hardworking engineers?")
print(return_OAI(pqt1))

```json
{
    "tables": ["Engineers", "Specialties"]
}
```


- Generate the Second Query - aggregated result

In [83]:
pqt3 = prompt_question_tables.format(tables=text_tables,
                                     question='Return the engineer with the highest number of completed projectse')
print(return_OAI(pqt3))

```json
{
    "Tables": ["Engineers", "Projects"]
}
```


# Version Three:  Aviation 

In [84]:
import pandas as pd

# Table and definitions 
data = {
    'table': ['Aircraft', 'Pilots', 'Flights', 'Maintenance'],
    'definition': [
        'Aircraft details: model, manufacturer, capacity, registration number',
        'Pilot details: name, license type, flight hours, assigned aircraft',
        'Flight details: flight number, origin, destination, schedule, status',
        'Maintenance records: aircraft ID, last service date, next due service, issues reported'
    ]
}

df = pd.DataFrame(data)
df.head()

,table,definition
0,Aircraft,"Aircraft details: model, manufacturer, capacit..."
1,Pilots,"Pilot details: name, license type, flight hour..."
2,Flights,"Flight details: flight number, origin, destina..."
3,Maintenance,"Maintenance records: aircraft ID, last service..."


-  Convert Table Definitions to Text Format

In [85]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])
print(text_tables)

Aircraft: Aircraft details: model, manufacturer, capacity, registration number
Pilots: Pilot details: name, license type, flight hours, assigned aircraft
Flights: Flight details: flight number, origin, destination, schedule, status
Maintenance: Maintenance records: aircraft ID, last service date, next due service, issues reported


- Generate and Execute the First Query

In [86]:
pqt1 = prompt_question_tables.format(tables=text_tables, question="Return the issue reported of the Airbus 320")
print(return_OAI(pqt1))


```json
{
    "Tables": ["Aircraft", "Maintenance"]
}
```


- Generate the Second Query - aggregated result


In [87]:
pqt3 = prompt_question_tables.format(tables=text_tables, question='Return the pilot with the highest number of flight hours')
print(return_OAI(pqt3))

```json
{
    "tables": ["Pilots"]
}
```
